In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd

from CTABGAN.model.ctabgan import CTABGAN
import pickle

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
dataset = 'Chicago'

In [4]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [5]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


In [6]:
if dataset is 'Chicago':
    discrete_columns = [
        'choice',
        'travel_dow',
        'trip_purpose',
        'hh_vehicles',
        'hh_size',
        'hh_bikes',
        'hh_descr',
        'hh_income',
        'gender',
        'license',
        'education_level',
        'work_status'
    ]
elif dataset is 'LPMC':
    discrete_columns = [
        'travel_mode',
        'purpose',
        'fueltype',
        'faretype',
        'bus_scale',
        'travel_year',
        'travel_month',
        'travel_date',
        'day_of_week',
        'female',
        'driving_license',
        'car_ownership',
        'pt_n_interchanges',
        'cost_driving_con_charge'
    ]

In [7]:
output_folder = '../output/TEST/CTABGAN/'

# Train the model

In [8]:
if dataset is 'Chicago':
    synthesizer =  CTABGAN(raw_csv_path = '../data/{}/data.csv'.format(dataset),
                     test_ratio = 0.2,
                     categorical_columns = discrete_columns, 
                     log_columns = [],
                     mixed_columns= {},
                     integer_columns = ['age'],
                     problem_type= {'Classification': 'choice'},
                     epochs = 2)
elif dataset is 'LPMC':
    synthesizer =  CTABGAN(raw_csv_path = '../data/{}/data.csv'.format(dataset),
                 test_ratio = 0.2,
                 categorical_columns = discrete_columns, 
                 log_columns = [],
                 mixed_columns= {'dur_pt_rail': [0.0], 'dur_pt_bus': [0.0], 'dur_pt_int': [0.0], 'cost_transit': [0.0, 1.5]},
                 integer_columns = ['age', 'distance'],
                 problem_type= {'Classification': 'travel_mode'},
                 epochs = 2)

In [9]:
synthesizer.fit()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.48s/it]

Finished training in 8.012378215789795  seconds.


In [10]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(synthesizer, handle)

In [8]:
with open('filename.pickle', 'rb') as handle:
    synth = pickle.load(handle)

In [9]:
syn = synth.generate_samples()

In [13]:
syn.to_csv('../synth_data/TEST/CTABGAN.csv', index= False)